## 일급 함수
* ### 런타임에 생성할 수 있다
* ### 데이터 구조체의 변수나 요소에 할당할 수 있다
* ### 함수 인수로 전달할 수 있다
* ### 함수 결과로 반환할 수 있다

In [4]:
import inspect


def factorial(n):
    '''returns n'''
    return 1 if n<2 else n * factorial(n-1)

fact = factorial # 함수를 호출한 것이 아닌 함수를 fact 변수에 할당
fact # <function __main__.factorial(n)>
fact(5) # 120

map(factorial,range(11)) # <map at 0x16b851abc40> -> factorial 함수를 0부터 10까지의 숫자에 적용한 결과를 반환하는 map 객체를 생성합니다. map 객체는 이터레이터이므로, 직접 출력하면 메모리 주소가 나타납니다.


list(map(fact,range(11)))
# list(map(fact, range(11)))는 0부터 10까지의 각 숫자에 대해 factorial 함수를 적용한 결과를 리스트로 변환하여 [1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]을 반환합니다.

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

## 고위 함수
* ### 함수를 인수로 받거나, 함수를 결과로 반환하는 함수를 고위함수 ex) map함수, sort함수


In [7]:
fruits = ['strawberry','fig','apple','cherry','raspberry','banana']
sorted(fruits,key=len)

# 람다를 이용한 역순정렬
sorted(fruits,key=lambda word:word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

In [ ]:
import random

class BingoCage:
    def __init__(self,items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
        
    def __call__(self):
        return self.pick()
    
    
bingo = BingoCage(range(3))
bingo()
bingo()
bingo() 


## 일급 함수 디자인 패턴
* ### 일급 함수를 지원하는 언어에서는 전략,명령,템플릿 메서드,비지터 패턴을 눈여겨 봐야함

In [ ]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

class LineItem:
    def __init__(self,product,quantity,price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order:
    def __init__(self,customer,cart,promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion

    def total(self):
        if not hasattr(self,'_total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due:{: 2f}>'
        return fmt.format(self.total(),self.due())

class Promotion(ABC): # 전략:추상 베이스 클래스
    @abstractmethod
    def discount(self,order):
        """할인액을 구체적인 숫자로 반환한다"""

class FidelityPromo(Promotion): # 전략객체
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""

    def discount(self,order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""

    def discount(self,order): # 전략객체
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

class LargeOrderPromo(Promotion): # 전략객체
    """10종류 이상의 상품을 구입하면 10% 할인 적용"""

    def discount(self,order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total()* .07
        return 0

joe = Customer('John Doe',0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple',10, 1.5),
        LineItem('watermellon',5,5.0)]
o1 = Order(joe,cart,FidelityPromo())
o2 = Order(ann,cart,FidelityPromo())
print(o1,o2)

* ### 함수지향 전략
* ### 전략 객체는 상태(객체 속성)를 가지고 있지 않음 
* ### 플라이웨이트 : 여러 콘텍스트에서 동시에 사용할 수 있는 공유 객체
* ### 함수는 파이썬이 모듈을 컴파일할 떄 단 한번만 생성되므로 플라이웨이트가 필요없음
* ### 일반 함수도 '여러 콘텍스트에서 동시에 공유할 수 있는 공유 객체'임

In [2]:
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

class LineItem:
    def __init__(self,product,quantity,price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order:  # Context
    def __init__(self,customer,cart,promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion # 할인 전략 주입

    def total(self):
        if not hasattr(self,'_total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due:{: 2f}>'
        return fmt.format(self.total(),self.due())

def fidelity_promo(order): # 전략객체
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order): # 전략객체
    """10종류 이상의 상품을 구입하면 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total()* .07
    return 0

<Order total: 10.00 due: 9.300000>

* ## 최선의 전략 선택하기 : 단순한 접근법
* ### 문제점 : 새로운 할인 전략을 추가하려면 함수를 코딩하고 이 함수를 promos 리스트에 추가해야함 아니면 새로운 할인 함수를 Order 객체에 인수로 전달해서 작동시킬 수 있지만, 이때 best_promo()는 새로운 할인 함수를 고려하지 않는다 

In [ ]:
promos = [fidelity_promo,bulk_item_promo,large_order_promo]

def best_promo(order):
    """최대로 할인받을 금액을 반환한다"""
    return max(promo(order) for promo in promos)

joe = Customer('John Doe', 0)
long_order = [LineItem(str(item_code),1,1.0)
              for item_code in range(10)]


Order(joe,long_order,best_promo)

* ## 모듈에서 전략 찾기
* ### 파이썬 모듈도 일급 객체
* ### globals() 내장 함수를 활용하여 best_promo()가 자동으로 다른 promo함수를 찾아내게 한다

In [3]:
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']

def best_promo(order):
    return max(promo(order) for promo in promos)

joe = Customer('John Doe', 0)
long_order = [LineItem(str(item_code),1,1.0)
              for item_code in range(10)]


Order(joe,long_order,best_promo)

<Order total: 10.00 due: 9.300000>

In [ ]:
# 새로운 promotinos 모듈을 내부 조사해서 만든 promos 리스트
# promos = [func for name, func in 
#           inspect.getmembers(promotions,inspect.isfunction)]
# 
# def best_promo(order):
#     return max(promo(order) for promo in promos)


"""
함수를 인수로 전달하는 기법을 사용하면 명령 디자인 패턴도 구현을 단순화 할 수 있음
- Command 패턴의 목적 : 연산을 실행하는 객체(호출차, Invoker)와 연산을 구현하는 객체(수신자, Receiver)를 분리함
- 명령 객체를 수신자와 호출자 사이에 놓고, 명령은 execute() 라는 단 하나의 메서드로 인터페이스를 구현함
- Command 객체 대신 간단히 함수를 바로 지정할 수 있음, 호출자는 command()로 사용할 수 있음. MacroCommand의 객체는 콜러블이 됨
"""

class Menu : 
    
    def __init__(self, macro_command) : 
        self.macro_command = macro_command
        
    def show_total_order(self) :
        print("== show total command!! ==")
        self.macro_command()
        

class MacroCommand : 
    """명령 리스트를 실행하는 명려"""
    
    def __init__(self, commands) :
        self.commands = list(commands)
        
    def __call__(self) : 
        for c in self.commands : 
            c()
            
# 하나의 Command
def order_pizza_command() : 
    print("order pizza")
    
def order_chiken_command() : 
    print("order chiken")
    
def order_coke_command() :
    print("order coke")
    
commands = [globals()[name] for name in globals() if name.endswith('_command') ]
macro = MacroCommand(commands)
menu = Menu(macro)
menu.show_total_order()

* ## 함수 데커레이터와 클로저
* ### 데커레이터 : 다른 함수를 인수로 받는 콜러블(데커레이트된 함수)이다.
* ### 데커레이터는 편리 구문일 뿐 / 런타임에 프로그램 행위를 변경하는 메타프로그래밍을 할때 데커레이터는 상당히 편리
* ### 데커레이터는 데커레이트된 함수를 다른 함수로 대체하는 능력이 있다
* ### 데커레이터는 모듈이 로딩될 떄 바로 실행된다

In [7]:
# @decorate
# def target():
#     print('running target()')
#     
# def target():
#     print('running target()')
# target = decorate(target)


def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')
    
target()
target # target이 inner()를 가르키고 있음

running inner()


<function __main__.deco.<locals>.inner()>

* ## 파이썬이 데커레이터를 실행하는 시점
* ### 데커레이터의 핵심 특징은 데커레이트된 함수가 정의된 직후에 실행된다는 것 즉, 임포트 타임에 실행된다
* 